In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import datetime
import numpy as np
np.random.seed(0)

In [ ]:
cols = ["Date", "Win_Score", "Win_Team", "Lose_Score", "Lose_Team", "Winning_Pitcher", "W_Pitcher_Wins", "W_Pitcher_Losses", "Losing_Pitcher", "L_Pitcher_Wins", "L_Pitcher_Losses", "Win_W%", "Win_RS", "Win_RA", "Win_Pythag", "Lose_W%", "Lose_RS", "Lose_RA", "Lose_Pythag"]
conf = ['AL', 'NL']
cols2 = ["Team", "W%", "RS", "RA", "Pythag"]
abv = ['ARI','ATL','BAL','BOS','CHC','CHW','CIN','CLE','COL','DET','HOU','KCR','LAA','LAD','MIA','MIL','MIN','NYM','NYY','OAK','PHI','PIT','SDP','SEA','SFG','STL','TBR','TEX','TOR','WSN']
teams_full = ['Arizona Diamondbacks','Atlanta Braves','Baltimore Orioles','Boston Red Sox','Chicago Cubs','Chicago White Sox','Cincinnati Reds','Cleveland Indians','Colorado Rockies','Detroit Tigers','Houston Astros','Kansas City Royals','LA Angels of Anaheim','Los Angeles Dodgers','Miami Marlins','Milwaukee Brewers','Minnesota Twins','New York Mets','New York Yankees','Oakland Athletics','Philadelphia Phillies','Pittsburgh Pirates','San Diego Padres','Seattle Mariners','San Francisco Giants','St. Louis Cardinals','Tampa Bay Rays','Texas Rangers','Toronto Blue Jays','Washington Nationals']
data = pd.DataFrame(None, columns=cols)
years = list(range(2010,2019))

for year in years:
    base = datetime.date(year,9,15)
    date_list = [base - datetime.timedelta(days=x) for x in range(0, 154)]

    #Loop through each day from the given date list
    for d in date_list:
        day = d.day
        month = d.month
        year = d.year
        date = str(year) + '-' + str(month) + '-' + str(day)  
        link = 'https://www.baseball-reference.com/boxes/?month=' +str(month) + '&day=' + str(day) + '&year=' + str(year)
        html_page = requests.get(link) #Make a get request to retrieve the page
        soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
  
        games = soup.find_all("div", class_="game_summary nohover")
    
        ts_al = pd.DataFrame(None, columns=cols2)
        ts_nl = pd.DataFrame(None, columns=cols2)
        tables = [ts_al, ts_nl]
               
        for c in [0,1]:

            stats = soup.find_all("div", id="all_standings-upto-" + conf[c] + "-overall")
            new = stats[0].find_all("tbody")[0]

            v1 = 2
            v2 = 4
            v3 = 5
            v4 = 6

            for t in list(range(0,14)):

                team = new("th")[t].text
                win_perc = new.find_all('td')[v1].text
                rs = new.find_all('td')[v2].text
                ra = new.find_all('td')[v3].text
                pythag = new.find_all('td')[v4].text

                tables[c] = tables[c].append({'Team': team,
                                        'W%': win_perc,
                                        'RS': rs,
                                        'RA': ra,
                                        'Pythag': pythag},
                                        ignore_index=True)

                v1 += 7
                v2 += 7
                v3 += 7
                v4 += 7

        ts_al = tables[0]
        ts_nl = tables[1]

        for t in list(range(0, len(abv))):
            try:
                ts_nl.loc[ts_nl['Team'] == abv[t], 'Team'] = teams_full[t]
            except:
                pass
            try:
                ts_al.loc[ts_al['Team'] == abv[t], 'Team'] = teams_full[t]
            except:
                pass
        
        # Loop through each game for a single day
        for g in list(range(0,len(games))):
            try:
                print(date)
                # Collect winning score
                winner = games[g].find_all("tr", class_="winner")
                strip_winner_score = winner[0].find_all("td", class_="right")
                winner_score = strip_winner_score[0].contents

                # Collect winning team
                winner = games[g].find_all("tr", class_="winner")
                winner_team = winner[0].find_all("td")[0].text

                # Collect losing score
                loser = games[g].find_all("tr", class_="loser")
                strip_loser_score = loser[0].find_all("td", class_="right")
                loser_score = strip_loser_score[0].contents

                # Collect losing team
                loser = games[g].find_all("tr", class_="loser")
                loser_team = loser[0].find_all("td")[0].text

                # Collect winning pitcher and record
                w_pitcher = games[g].find_all("td")[7].contents

                if w_pitcher == None:
                    w_name = ["NULL","NULL","NULL"]
                    w_record = ["NULL","NULL","NULL"]
                else:
                    w_name = w_pitcher[0].split("(")
                    w_name = w_name[:-1]
                    w_record = w_pitcher[0].split("(")
                    w_record = w_record[1].replace(")","")
                    w_record = w_record.split("-")

                # Collect losing pitcher and record
                l_pitcher = games[g].find_all("td")[9].contents

                if l_pitcher == None:
                    l_name = ["NULL","NULL","NULL"]
                    l_record = ["NULL","NULL","NULL"]
                else:   
                    l_name = l_pitcher[0].split("(")
                    l_name = l_name[:-1]
                    l_record = l_pitcher[0].split("(")
                    l_record = l_record[1].replace(")","")
                    l_record = l_record.split("-")

                values = [int(winner_score[0]), winner_team, int(loser_score[0]), loser_team]

                # Add data to dataframe
                data = data.append({'Date': date,
                                'Win_Score': values[0],
                                'Win_Team': values[1],
                                'Lose_Score': values[2],
                                'Lose_Team': values[3],
                                'Winning_Pitcher': w_name[0],
                                'W_Pitcher_Wins': w_record[0],
                                'W_Pitcher_Losses': w_record[1],
                                'Losing_Pitcher': l_name[0],
                                'L_Pitcher_Wins': l_record[0],
                                'L_Pitcher_Losses': l_record[1]},
                                ignore_index=True)
                
                cols3 = ["NULL", "Win_W%", "Win_RS", "Win_RA", "Win_Pythag"]
                cols4 = ["NULL", "Lose_W%", "Lose_RS", "Lose_RA", "Lose_Pythag"]
                for team in teams_full:
                    for i in list(range(0,14)):
                        if ts_nl['Team'][i] == team:
                            for c in list(range(1,5)):
                                x = ts_nl.loc[ts_nl['Team'] == team, cols2[c]]
                                x = float(x)
                                data.loc[data['Win_Team'] == team, cols3[c]] = x
                                
                                y = ts_nl.loc[ts_nl['Team'] == team, cols2[c]]
                                y = float(x)
                                data.loc[data['Lose_Team'] == team, cols4[c]] = x
                            
                        if ts_al['Team'][i] == team:
                            for c in list(range(1,5)):
                                x = ts_al.loc[ts_al['Team'] == team, cols2[c]]
                                x = float(x)
                                data.loc[data['Win_Team'] == team, cols3[c]] = x
                                
                                y = ts_al.loc[ts_al['Team'] == team, cols2[c]]
                                y = float(x)
                                data.loc[data['Lose_Team'] == team, cols4[c]] = x
                            
                
            except:
                pass
            
data.to_excel("MLB-Game-Data-Processed.xlsx")

In [ ]:
link = ''
data = pd.read_excel(link)

In [ ]:
years = list(range(2010,2019))
g = []
cols3 = ["Date", "Home_Team", "Away_Team"]
df2 = pd.DataFrame(None, columns=cols3)

for year in years:
    print(year)
    link = 'https://www.baseball-reference.com/leagues/MLB/' + str(year) + '-schedule.shtml'
    html_page = requests.get(link) #Make a get request to retrieve the page
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
  
    games = soup.find_all("div", class_="section_content")
    
    for i in list(range(0,len(games[0].find_all('a')))):

        if games[0].find_all('a')[i].text[0] == '2':

            date = games[0].find_all('a')[i].text[0:10]

            for i in list(range(0,len(g),2)):
                df2 = df2.append({'Date': date,
                                  'Away_Team': g[i],
                                  'Home_Team': g[i+1]},
                                  ignore_index=True)

            g.clear()

        elif games[0].find_all('a')[i].text[0:4] == 'Boxs':
            pass

        else:
            g.append(games[0].find_all('a')[i].text)

for d in list(range(len(df2))):
    year = df2.Date[d][0:4]
    if df2.Date[d][5:7][0] == '0':
        month = df2.Date[d][6:7]
    else:
        month = df2.Date[d][5:7]
    if df2.Date[d][8:10][0] == '0':    
        day = df2.Date[d][9:10]
    else:
        day = df2.Date[d][8:10]
    date = month + '-' + day + '-' + year
    df2.Date[d] = date

In [ ]:
merged_home_winners_df = pd.merge(data, df2, left_on=['Date', 'Win_Team'], right_on=['Date', 'Home_Team'])
merged_away_winners_df = pd.merge(data, df2, left_on=['Date', 'Win_Team'], right_on=['Date', 'Away_Team'])
final_df = pd.concat([merged_home_winners_df, merged_away_winners_df])
final_df.reset_index(inplace=True)

for i in list(range(0, len(df))):
    if final_df['Win_Team'][i] == final_df['Home_Team'][i]:
        final_df['Home_Team'][i] = 'W'
        final_df['Away_Team'][i] = 'L'
    else:
        final_df['Home_Team'][i] = 'W'
        final_df['Away_Team'][i] = 'L'

final_df['Run_Diff'] = final_df['Win_Score'] - final_df['Lose_Score']

final_df.to_excel("MLB-Game-Data-Complete.xlsx")

In [2]:
link2 = ''
df = pd.read_excel(link2)
df.head()

,Date,Win_Score,Win_Team,Lose_Score,Lose_Team,Winning_Pitcher,W_Pitcher_Wins,W_Pitcher_Losses,Losing_Pitcher,L_Pitcher_Wins,...,Win_RS,Win_RA,Win_Pythag,Lose_W%,Lose_RS,Lose_RA,Lose_Pythag,Home_Team,Away_Team,Run_Diff
0,4-15-2010,8,Minnesota Twins,0,Boston Red Sox,Francisco Liriano,1,0,Tim Wakefield,0,...,38,63,0.284,0.167,46,79,0.271,W,L,8
1,4-15-2010,6,Los Angeles Dodgers,5,Arizona Diamondbacks,Jonathan Broxton,1,0,Blaine Boyer,1,...,48,68,0.346,0.417,66,75,0.442,W,L,1
2,4-15-2010,6,Oakland Athletics,2,Baltimore Orioles,Ben Sheets,1,0,David Hernandez,0,...,47,52,0.454,0.500,46,53,0.436,W,L,4
3,4-15-2010,7,Toronto Blue Jays,3,Chicago White Sox,Dana Eveland,2,0,Freddy Garcia,0,...,68,50,0.637,0.538,76,65,0.571,W,L,4
4,4-15-2010,6,New York Yankees,2,LA Angels of Anaheim,Phil Hughes,1,0,Scott Kazmir,0,...,66,61,0.536,0.615,49,44,0.549,W,L,4
